In [ ]:
import numpy as np
import sympy as sy
import holoviews as hv; hv.extension('bokeh')
import panel as pn; pn.extension()

# Fourier Transforms

In [ ]:
def define_w(n):
    return sy.exp(2j*sy.pi/n)

def FourierMatrix(n):
    w = define_w(n)
    l = sy.Matrix( [[ (w**p)**k for k in range(n)] for p in range(n)] )
    return l

def numFourierMatrix(n):
    w = np.cos(2*np.pi/n)+1j*np.sin(2*np.pi/n)  # convert to floating point
    l = np.array( [[ w**(p*k) for k in range(n)] for p in range(n)] )*1/np.sqrt(n)
    return l

F = FourierMatrix(4)   # multiply by 1/sqrt(n) to make the matrix unitary. See below
F

The Fourier Matrix $F$ is an invertible matrix defined  from a term $e^{i \frac{2 \pi}{n}}$

The inverse of the Fourier Matrix is it's hermitian transpose: $F^{-1} = \bar{F}^t = F^H$

In [ ]:
FFh = F*F.T.conjugate()                  # F times its Hermitian Transpose
FFh.applyfunc(lambda x: round(x.n(), 3)) # no surprise: we just need to scale M or its hermitian transpose
                                         # by convention, we scale both F and F^h by 1/sqrt(n)

The Fourier Matrix $F$ is invertible. Computing  

Let's set up a function of time

In [ ]:
N = 256
t = np.linspace(0, 1, N)
x = 2*np.sin(2*np.pi*10*t)   # period of the sine is 1/10 secs
pn.Row(hv.Curve((t,x)).opts(title="Function  y = 2 sin( 2pi f t)"))

If we think of $x$ as a vector, then $y = F x$ rewrites $x$ as a linear combination of the columns of $F^{-1}:\quad x = F^{-1} y$

$\quad$ This is just a **change of basis!**

What are the basis vectors?

In [ ]:
F  = numFourierMatrix(N)
Fh = F.T.conj()

def plot( basis_vector_index ):
    h = hv.Curve(np.real(Fh[:,basis_vector_index]), label='real part')*\
        hv.Curve(np.imag(Fh[:,basis_vector_index]), label='imag part')
    return h.opts( title=f'Basis Vector # {basis_vector_index}', width=700, legend_position='right')
pn.interact( plot, basis_vector_index=(0,N//2))

So the basis vectors are sines and cosines with increasing frequencies!

The x vector we computed above was one such sine.<br>
Let's compute the corresponding y vector:

In [ ]:
y = F @ x
pn.Row(hv.Curve(abs(y),label="magnitude of the component vector y").opts(width=700))

The reason we got two peaks is that we used a real vector x, i.e., not really a basis vector,<br>
but the difference of two basis vectors: index 10, and index 256-10

Actually drawing these plots as curves is misleading: we have a discrete set of values

In [ ]:
# What if we make the function more complicated?
x = 2.0*np.sin(2*np.pi*10*t) \
   +0.5*np.cos(2*np.pi*100*t)
y = F @ x
pn.Row( hv.Curve(x, label='x').opts(axiswise=True),  hv.Curve(abs(y),label="|y|"))

So we created an x vector composed of a sine with a superimposed wiggle with frequencies 10 and 100 Hz,<br>
and therefore, there are 4 entries in the transformed coefficient vector:<br> entries at 10 and 100, as well as 256-10, 256-100

In [ ]:
# What if we make the function more complicated?
x = 2.0*(0.5-t)**3 + 0.1*t \
   + 0.1*np.cos(2*np.pi*100*t)
y = F @ x
pn.Row( hv.Curve(x, label='x').opts(axiswise=True),  hv.Curve(abs(y),label="|y|"))

At first blush, this looks like 4 entries? It turns out, that there are a lot of small entries!<br>
This is easier to see on a log scale 

In [ ]:
print("Smallest coefficient is", min(abs(y)) )
pn.Row(hv.Curve(abs(y)).opts(logy=True, title = "coefficient vector abs.(y)"))

Hmm, what if we throw some of those small values out, (they are just small wiggles),<br>
and transform back
to our original basis? Is $\tilde{x} \approx F y$?

In [ ]:
y_approx = np.array( [ v if abs(v) > 0.05 else 0 for v in y])
pn.Row((hv.Curve( abs(y_approx), label='larger values only' ) *\
        hv.Curve( abs(y),        label='all values' )).opts( legend_position = 'right', width=700))

In [ ]:
x_approx    = Fh @ y_approx
x_recovered = Fh @ y
pn.Row(hv.Curve( np.real(x_approx - x_recovered), label='difference' ).opts(axiswise=True),
       hv.Curve( np.real(x_approx), label='approx' )*hv.Curve(np.real(x_recovered), label='recovered')
      )

Click on the magnifying glass to zoom

# Conclusion

The change of basis to Fourier Coefficients reveals Features of the original series<br>
that were not readily apparent.

We can analyze the function in either representation, and introduce transformations<br>
in one domain that may be much harder inthe other.

Note the clipping function we applied is non-linear...

The structure of the $F$ matrix can be exploited to greatly speed up the computation of $F x$<br>
(the Fast Fourier Transform)